# Another Instagram Filter (AI-Filter)

## Import Dependencies

In [2]:
import cv2
import mediapipe as mp
import math
from math import hypot
import numpy as np
from statistics import mean

##  Declare functions

In [20]:
def sepia(img):
    img_sepia = np.array(img, dtype=np.float64) # converting to float to prevent loss
    img_sepia = cv2.transform(img_sepia, np.matrix([[0.272, 0.534, 0.131],
                                    [0.349, 0.686, 0.168],
                                    [0.393, 0.769, 0.189]])) # multipying image with special sepia matrix
    img_sepia[np.where(img_sepia > 255)] = 255 # normalizing values greater than 255 to 255
    img_sepia = np.array(img_sepia, dtype=np.uint8)
    return img_sepia

def changeBright(img, beta_value ):
    img_bright = cv2.convertScaleAbs(img, beta=beta_value)
    return img_bright

## Video Feed

In [48]:
cap = cv2.VideoCapture(0) 
 
mpHands = mp.solutions.hands #detects hand/finger
hands = mpHands.Hands()   #complete the initialization configuration of hands
mpDraw = mp.solutions.drawing_utils

volbar=400
volper=0

angle_max = 250
k = 1.8
angle = 120
length = 50
brightness = 0
r_dial = 50

while True:
    success,img = cap.read() #If camera works capture an image
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #Convert to rgb
    img = changeBright(img, brightness)
    #Collection of gesture information
    results = hands.process(imgRGB) #completes the image processing.
 
    ls_lm = [] #empty list
    if results.multi_hand_landmarks: #list of all hands detected.
        #By accessing the list, we can get the information of each hand's corresponding flag bit
        for handlandmark in results.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark): #adding counter and returning it
                # Get finger joint points
                h,w,_ = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                ls_lm.append([id,cx,cy]) #adding to the empty list 'ls_lm'
            mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)
    
    if ls_lm != []:
        x_0,y_0 = ls_lm[0][1],ls_lm[0][2] # wrist
        x_4,y_4 = ls_lm[4][1],ls_lm[4][2] # thumb tip
        x_5,y_5 = ls_lm[5][1],ls_lm[5][2] # index finger mcp
        x_8,y_8 = ls_lm[8][1],ls_lm[8][2] # index finger tip
        x_9,y_9 = ls_lm[9][1],ls_lm[9][2] # middle finger mcp
        x_12,y_12 = ls_lm[12][1],ls_lm[12][2] # middle finger tip
        x_16,y_16 = ls_lm[16][1],ls_lm[16][2] # ring finger tip
        x_20,y_20 = ls_lm[20][1],ls_lm[20][2] # pinky finger tip

        x_palm = x_9#int(x_0 + (x_4-x_0)/3)
        y_palm = y_9#int(y_5 + (y_0-y_5)/2.3)
        
        #creating circle at the tips of thumb and index finger
        cv2.circle(img,(x_4,y_4),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.circle(img,(x_8,y_8),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        
        x_hat = x_0-x_8
        z_hat = hypot(x_0-x_8,y_0-y_8) + 1
        angle_hat = math.degrees(np.arcsin(x_hat/z_hat))
        
        s_4 = hypot(x_4-x_palm, y_4-y_palm)
        s_8 = hypot(x_8-x_palm, y_8-y_palm)
        s_12 = hypot(x_12-x_palm, y_12-y_palm)
        s_16 = hypot(x_16-x_palm, y_16-y_palm)
        s_20 = hypot(x_20-x_palm, y_20-y_palm)
        
        s_mean = mean([s_4, s_8, s_12, s_16, s_20])
        
        if (s_mean <= r_dial + 10) and (s_mean >= r_dial -10): 
            cv2.circle(img, (x_palm, y_palm ),r_dial,(255,0,255), 5)
            length = int((angle_hat*2.8/angle_max)*250)
            brightness = np.interp(length,[0,250],[-100,100])
            img = changeBright(img, brightness)
        
        volbar=np.interp(length,[0,250],[350,100])
        volper=np.interp(length,[0,250],[0,100])

    cv2.rectangle(img,(550,100),(600,350),(0,0,255,0.1),4)
    cv2.rectangle(img,(550,int(volbar)),(600,350),(255,0,255,0.3),cv2.FILLED)
    cv2.putText(img,f"Brightness: {int(volper)}%",(10,40),cv2.FONT_ITALIC,1,(0, 255, 120),3)
    cv2.imshow('Image',img)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break
        
cap.release()     #stop cam       
cv2.destroyAllWindows() #close window

### Stop Camera and Close All Windows

Run this cell in case the video capture bugs and you can't close the image window

In [27]:
cap.release()  
cv2.destroyAllWindows()